In [1]:
import keras
import numpy as np
from sklearn.model_selection import train_test_split

2024-11-09 15:12:05.620342: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-09 15:12:05.933488: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
NO_BARK_BEETLE = "dataset/no_bark_beetle.npy"
BARK_BEETLE = "dataset/bark_beetle.npy"

In [3]:
no_beetle_data = np.load(NO_BARK_BEETLE)
no_beetle_labels = np.zeros(no_beetle_data.shape[0])

beetle_data = np.load(BARK_BEETLE)
beetle_labels = np.ones(beetle_data.shape[0])


data = np.concatenate((beetle_data, no_beetle_data))
labels = np.concatenate((beetle_labels, no_beetle_labels))


In [ ]:
labels = np.expand_dims(labels, axis=1)

In [4]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, shuffle=True)

In [5]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(79, 31, 14, 10)
(20, 31, 14, 10)
(79,)
(20,)


In [6]:
# create and fit the LSTM network
model = keras.Sequential()
model.add(keras.layers.Input(shape=(31, 14, 10)))
model.add(keras.layers.TimeDistributed(keras.layers.LSTM(48)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(24, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 31, 48)         │        11,328 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1488)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 24)             │        35,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,089 (183.94 KB)

 Trainable params: 47,089 (183.94 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
metric = keras.metrics.BinaryAccuracy()
model.compile(
    optimizer=keras.optimizers.Adam(0.001), loss="binary_crossentropy", metrics=[metric]
)

In [9]:
model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=5
)

Epoch 1/5


2024-11-09 15:13:06.295932: E tensorflow/core/util/util.cc:131] oneDNN supports DT_UINT8 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


3/3 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - binary_accuracy: 0.5986 - loss: 0.6533 - val_binary_accuracy: 0.8500 - val_loss: 0.5684
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - binary_accuracy: 0.9693 - loss: 0.3933 - val_binary_accuracy: 0.8500 - val_loss: 0.5498
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - binary_accuracy: 0.9693 - loss: 0.2402 - val_binary_accuracy: 0.8500 - val_loss: 0.5604
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - binary_accuracy: 0.9756 - loss: 0.1858 - val_binary_accuracy: 0.8500 - val_loss: 0.5995
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - binary_accuracy: 0.9937 - loss: 0.1034 - val_binary_accuracy: 0.8500 - val_loss: 0.6567


In [10]:
model.save("../model.keras")